In [120]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.pyplot import rcParams
import seaborn as sbs
import scipy as sp
from scipy import stats
import statistics as st  
import datetime 
from datetime import datetime      
from datetime import timedelta
#date_parser=lambda x:datetime.strptime(x,'%y/%m/%d')

In [121]:
from math import sqrt
from numpy import split
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM 
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime
from keras import optimizers
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from numpy import savetxt
import seaborn as sbs

In [122]:
data=pd.read_csv("C:/Users/georgea1/OneDrive - Trinity College Dublin/Angel George/Chap 2. Treatment/Data/LSTM Data/First Relapse_August/LSTM_First_relapse_12_months_data1_21_08_2024.csv")
data

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
0,1,2012-08-31,0.0,MPO,NaN,7.0,0,Not received,1,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,2012-09-07,0.2,MPO,7.0,10.0,0,Not received,1,0,...,NaN,No,Functioning native kidneys,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,2012-09-17,0.6,MPO,10.0,4.0,0,Not received,1,0,...,NaN,No,Functioning native kidneys,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,2012-09-21,0.7,MPO,4.0,7.0,0,Not received,1,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,2012-09-28,0.9,MPO,7.0,3.0,0,Not received,1,0,...,NaN,No,Functioning native kidneys,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0
7017,28615,2017-02-24,71.1,MPO,647.0,2181.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,1,NaN,Negative,178.0,NaN,0
7018,28615,2023-02-14,142.7,MPO,2181.0,357.0,0,Not received,1,0,...,no,Yes,Off Dialysis,NaN,>=+3,76.0,>=+3,214.0,9.0,1
7019,28615,2024-02-06,154.5,MPO,357.0,104.0,0,Not received,1,0,...,no,Yes,Off Dialysis,NaN,>=+3,NaN,>=+3,314.0,21.0,1


In [123]:
data['CPD_on_off'].value_counts(dropna=False)

CPD_on_off
On Treatment                4648
Off Treatment               1518
Manual review                417
Treatment Status Unknown     283
Prednisolone<=10             155
Name: count, dtype: int64

## Inclusion Criteria

In [124]:
data['CPD_relapse'].value_counts(dropna=False)

CPD_relapse
No Relapse                4046
NaN                       2298
Definite Relapse           413
Exclude this encounter     205
Possible Relapse            59
Name: count, dtype: int64

In [125]:

data.drop(data[data['CPD_relapse']=='Exclude this encounter'].index,inplace=True)


In [126]:
data['CPD_relapse'].value_counts(dropna=False)

CPD_relapse
No Relapse          4046
NaN                 2298
Definite Relapse     413
Possible Relapse      59
Name: count, dtype: int64

In [127]:
data.shape

(6816, 33)

In [128]:
data['RKD.ID'].nunique()

595

## Exclude encounters during the induction phase

In [129]:
data = data[data['Interval.from.diagnosis..months._x'] >=6]
data

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0
7017,28615,2017-02-24,71.1,MPO,647.0,2181.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,1,NaN,Negative,178.0,NaN,0
7018,28615,2023-02-14,142.7,MPO,2181.0,357.0,0,Not received,1,0,...,no,Yes,Off Dialysis,NaN,>=+3,76.0,>=+3,214.0,9.0,1
7019,28615,2024-02-06,154.5,MPO,357.0,104.0,0,Not received,1,0,...,no,Yes,Off Dialysis,NaN,>=+3,NaN,>=+3,314.0,21.0,1


In [130]:
data['RKD.ID'].nunique()

595

In [131]:
data['CPD_relapse'].value_counts()

CPD_relapse
No Relapse          4046
Definite Relapse     413
Possible Relapse      59
Name: count, dtype: int64

## Check for duplicate encounters

In [132]:
## Remove duplicates:
duplicateRows = data[data.duplicated(['RKD.ID','Date_Of_Visit'])]
duplicateRows

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months


## Remove first relapse encounters and the encounters after first relapse

In [133]:
# Create the 'rlp' column with 1 for 'Definite Relapse' and NaN for others
data['rlp'] = data['CPD_relapse'].apply(lambda x: 1 if x == 'Definite Relapse' else np.nan)

# Forward fill the 'rlp' column within each 'RKD.ID' group
data['rlp'] = data.groupby('RKD.ID')['rlp'].ffill()



C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3302478314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rlp'] = data['CPD_relapse'].apply(lambda x: 1 if x == 'Definite Relapse' else np.nan)
C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3302478314.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rlp'] = data.groupby('RKD.ID')['rlp'].ffill()


In [134]:

data

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months,rlp
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0,NaN
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0,NaN
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0,NaN
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0,NaN
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0,NaN
7017,28615,2017-02-24,71.1,MPO,647.0,2181.0,0,Not received,1,0,...,No,Off Dialysis,NaN,1,NaN,Negative,178.0,NaN,0,NaN
7018,28615,2023-02-14,142.7,MPO,2181.0,357.0,0,Not received,1,0,...,Yes,Off Dialysis,NaN,>=+3,76.0,>=+3,214.0,9.0,1,1.0
7019,28615,2024-02-06,154.5,MPO,357.0,104.0,0,Not received,1,0,...,Yes,Off Dialysis,NaN,>=+3,NaN,>=+3,314.0,21.0,1,1.0


In [135]:
# Drop rows where 'rlp' is equal to 1
data = data[data['rlp'] != 1]


In [136]:
relapse=data.loc[(data['Relapse_next_12_months']==1)]
relapse

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months,rlp
37,1,2020-05-22,92.7,MPO,280.0,189.0,0,Not received,1,0,...,Yes,Off Dialysis,151.8,NaN,72.0,NaN,109.0,2.0,1,NaN
38,1,2020-11-27,98.9,MPO,189.0,154.0,0,Not received,1,0,...,Yes,Off Dialysis,171.4,>=+3,85.0,Negative,115.0,3.0,1,NaN
134,3,2016-07-08,51.2,MPO,154.0,14.0,0,Not received,0,1,...,Yes,Off Dialysis,119.5,2,51.0,Negative,118.0,3.0,1,NaN
135,3,2016-07-22,51.7,MPO,14.0,14.0,0,Not received,0,1,...,Yes,Off Dialysis,NaN,2,42.0,Negative,122.0,4.0,1,NaN
136,3,2016-08-05,52.1,MPO,14.0,91.0,0,Not received,0,1,...,Yes,Off Dialysis,NaN,NaN,42.0,NaN,129.0,4.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6846,28388,2022-07-01,8.4,PR3,8.0,77.0,0,Not received,1,0,...,No,Off Dialysis,170.0,>=+3,94.0,>=+3,126.0,61.0,1,NaN
6876,28403,2021-11-12,26.1,PR3,795.0,151.0,0,Not received,0,0,...,No,Off Dialysis,NaN,Negative,3.0,Negative,85.0,1.0,1,NaN
6883,28409,2022-10-07,9.9,MPO,210.0,53.0,1,Dose Unknown,0,0,...,No,Off Dialysis,NaN,Negative,NaN,Negative,80.0,3.0,1,NaN
6901,28415,2021-08-06,8.8,PR3,176.0,66.0,1,Dose Unknown,0,0,...,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


In [137]:
data

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months,rlp
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0,NaN
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0,NaN
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0,NaN
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0,NaN
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
7012,28594,2024-01-05,10.0,PR3,18.0,NaN,1,Rituximab low,0,0,...,No,Off Dialysis,NaN,Negative,1.2,Negative,75.0,4.0,0,NaN
7014,28603,2024-02-11,42.5,PR3,3208.0,NaN,1,Dose Unknown,0,0,...,No,Off Dialysis,NaN,2,NaN,2,54.0,NaN,0,NaN
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0,NaN


In [138]:
data = data.drop(columns=['rlp'])

In [139]:
data.shape

(2652, 33)

In [140]:
data['RKD.ID'].nunique()

532

In [141]:
data['Relapse_next_12_months'].value_counts(dropna=False)

Relapse_next_12_months
0    2432
1     220
Name: count, dtype: int64

In [142]:
# Function to drop rows after first occurrence of Relapse_next_12_months == 1
def drop_after_relapse(group):
    index_to_drop = group[group['Relapse_next_12_months'] == 1].index
    if not index_to_drop.empty:
        first_index = index_to_drop[0]
        return group[:first_index + 1]  # Include the row with 1
    return group  # If no 1 found, keep the group as is

# Apply the function to each group (each RKD.ID)
data = data.groupby('RKD.ID', group_keys=False).apply(drop_after_relapse)


C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\355082902.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('RKD.ID', group_keys=False).apply(drop_after_relapse)


In [143]:
#data.to_csv('C:/Users/georgea1/OneDrive - Trinity College Dublin/Desktop/RNN2.csv',index=True)


## Patient Selection

In [144]:
##Total number of patients in the RIV registry after applying inclusion criteria
data['RKD.ID'].nunique()

532

In [145]:
## Total number of encounters after applying inclusion criteria
data.shape

(2652, 33)

## Exclude patients, if they had only one encounter after induction

In [146]:
# Group by RKD.ID and count the number of visits for each ID
visit_counts = data.groupby('RKD.ID').size()
visit_counts 

RKD.ID
1        28
2        41
3        14
4        15
6        17
         ..
28587     1
28592     2
28594     2
28603     1
28615     2
Length: 532, dtype: int64

In [147]:
observation_summary = pd.DataFrame({
    'Observation Count': visit_counts.value_counts().index,
    'Number of IDs': visit_counts.value_counts().values
})

# Categorize the counts
def categorize_observations(count):
    if count == 1:
        return '1 observation'
    elif count == 2:
        return '2 observations'
    elif count ==3:
        return '3 observations'
    elif count ==4:
        return '4 observations'   
    else:
        return 'more than 5 observations'


observation_summary['Category'] = observation_summary['Observation Count'].apply(categorize_observations)

# Summarize the counts in each category
final_summary = observation_summary.groupby('Category')['Number of IDs'].sum().reset_index()

# Display the summary
final_summary

,Category,Number of IDs
0,1 observation,129
1,2 observations,91
2,3 observations,61
3,4 observations,59
4,more than 5 observations,192


In [148]:
# Filter IDs with at least 3 observations
ids_to_keep = visit_counts[visit_counts >=2].index

# Subset the original DataFrame to keep only the filtered IDs
data = data[data['RKD.ID'].isin(ids_to_keep)]

data

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.7,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7012,28594,2024-01-05,10.0,PR3,18.0,NaN,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,Negative,1.2,Negative,75.0,4.0,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


## Relapse encounters

In [149]:
relapse=data.loc[(data['Relapse_next_12_months']==1)]
relapse

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
37,1,2020-05-22,92.7,MPO,280.0,189.0,0,Not received,1,0,...,no,Yes,Off Dialysis,151.8,NaN,72.0,NaN,109.0,2.0,1
38,1,2020-11-27,98.9,MPO,189.0,154.0,0,Not received,1,0,...,no,Yes,Off Dialysis,171.4,>=+3,85.0,Negative,115.0,3.0,1
134,3,2016-07-08,51.2,MPO,154.0,14.0,0,Not received,0,1,...,no,Yes,Off Dialysis,119.5,2,51.0,Negative,118.0,3.0,1
135,3,2016-07-22,51.7,MPO,14.0,14.0,0,Not received,0,1,...,no,Yes,Off Dialysis,NaN,2,42.0,Negative,122.0,4.0,1
136,3,2016-08-05,52.1,MPO,14.0,91.0,0,Not received,0,1,...,no,Yes,Off Dialysis,NaN,NaN,42.0,NaN,129.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6738,28319,2021-01-20,24.0,MPO,366.0,85.0,1,Dose Unknown,0,0,...,no,No,Off Dialysis,NaN,NaN,NaN,NaN,69.0,2.0,1
6761,28325,2020-06-29,260.9,PR3,231.0,168.0,0,Not received,0,1,...,no,Yes,Off Dialysis,NaN,Negative,NaN,Negative,69.0,1.6,1
6845,28388,2022-06-23,8.1,PR3,90.0,8.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,Negative,122.0,99.0,1
6846,28388,2022-07-01,8.4,PR3,8.0,77.0,0,Not received,1,0,...,no,No,Off Dialysis,170.0,>=+3,94.0,>=+3,126.0,61.0,1


In [150]:
#data.to_csv('C:/Users/georgea1/OneDrive - Trinity College Dublin/Desktop/PARADISE_including_induction.csv',index=True)

## Feature Selection

## ANCA Specificity

In [151]:
data['At.any.point.ANCA.specificity'].value_counts(dropna=False)

At.any.point.ANCA.specificity
MPO               1230
PR3               1199
ELISA negative      84
MPO and PR3         10
Name: count, dtype: int64

## Consider Elisa negative, MPO & PR3, No Elisa performed as 'Other'

In [152]:
data['At.any.point.ANCA.specificity'] = data['At.any.point.ANCA.specificity'].replace(
    ['ELISA negative', 'MPO and PR3', 'No ELISA performed'], 
    'Other'
)

In [153]:
data['At.any.point.ANCA.specificity'].value_counts(dropna=False)

At.any.point.ANCA.specificity
MPO      1230
PR3      1199
Other      94
Name: count, dtype: int64

In [154]:
mpo=data.loc[(data['At.any.point.ANCA.specificity']=='MPO')]
mpo

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,28546,2023-06-02,9.3,MPO,310.0,278.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,31.3,1,17.0,2,177.0,32.0,0
6993,28546,2024-03-06,18.5,MPO,278.0,100.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,4.0,NaN,147.0,NaN,0
6994,28546,2024-06-14,21.7,MPO,100.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,43.7,Negative,5.6,1,161.0,13.0,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [155]:
mpo['RKD.ID'].nunique()

205

In [156]:
pr3=data.loc[(data['At.any.point.ANCA.specificity']=='PR3')]
pr3

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
206,6,2012-11-16,211.5,PR3,6422.0,238.0,0,Not received,0,1,...,yes,No,Off Dialysis,NaN,NaN,NaN,NaN,75.0,1.0,0
207,6,2013-07-12,219.3,PR3,238.0,126.0,0,Not received,0,1,...,yes,No,Off Dialysis,NaN,NaN,1.0,NaN,70.0,1.0,0
208,6,2013-11-15,223.5,PR3,126.0,140.0,0,Not received,0,1,...,yes,No,Off Dialysis,NaN,NaN,20.0,NaN,68.0,1.0,0
209,6,2014-04-04,228.1,PR3,140.0,126.0,0,Not received,0,1,...,yes,No,Off Dialysis,NaN,NaN,NaN,NaN,75.0,1.0,0
210,6,2014-08-08,232.2,PR3,126.0,126.0,0,Not received,0,1,...,yes,No,Off Dialysis,NaN,NaN,NaN,NaN,68.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,28481,2024-05-10,17.4,PR3,15.0,NaN,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,32.4,Negative,0.6,Negative,82.0,6.0,0
7008,28592,2023-03-31,6.5,PR3,199.0,259.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.7,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


In [157]:
pr3['RKD.ID'].nunique()

189

In [158]:
oth=data.loc[(data['At.any.point.ANCA.specificity']=='Other')]
oth

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
1811,153,2017-10-13,7.5,Other,98.0,4.0,0,Not received,1,0,...,no,No,Off Dialysis,126.33,>=+3,NaN,>=+3,NaN,NaN,0
1812,153,2017-10-17,7.6,Other,4.0,129.0,0,Not received,1,0,...,no,No,Functioning native kidneys,119.91,NaN,NaN,NaN,NaN,NaN,0
1813,153,2018-02-23,11.8,Other,129.0,98.0,0,Not received,1,0,...,no,No,On Dialysis,89.04,>=+3,38.0,2,999.0,15.5,0
1814,153,2018-06-01,15.0,Other,98.0,84.0,0,Not received,1,0,...,no,No,On Dialysis,NaN,2,NaN,2,999.0,17.0,0
1815,153,2018-08-24,17.8,Other,84.0,105.0,0,Not received,1,0,...,no,No,On Dialysis,NaN,1,NaN,2,999.0,24.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,12078,2019-09-04,38.2,Other,358.0,NaN,0,Not received,1,1,...,NaN,No,On Dialysis,NaN,NaN,NaN,NaN,999.0,NaN,0
5350,12089,2017-06-19,7.8,Other,133.0,227.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,356.0,NaN,193.0,NaN,0
5351,12089,2018-02-01,15.2,Other,227.0,298.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,232.0,NaN,0
5352,12089,2018-11-26,25.0,Other,298.0,226.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,319.0,NaN,301.0,5.0,0


In [159]:
oth['RKD.ID'].nunique()

9

## Rituximab : Patients who received RTX during induction phase

In [160]:
data['RTX_Status_induction'].value_counts(dropna=False)

RTX_Status_induction
0    1936
1     587
Name: count, dtype: int64

In [161]:
rtx_yes=data.loc[(data['RTX_Status_induction']==1)]
rtx_yes

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
602,33,2016-02-02,15.4,MPO,361.0,412.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,62.0,60.0,0
603,33,2017-03-20,28.9,MPO,412.0,73.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,73.0,1.7,0
604,33,2017-06-01,31.3,MPO,73.0,852.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
605,33,2019-10-01,59.3,MPO,852.0,227.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,10.0,NaN,64.0,3.0,0
606,33,2020-05-15,66.8,MPO,227.0,47.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,160.56,Negative,NaN,Negative,61.0,91.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,28546,2024-06-14,21.7,MPO,100.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,43.70,Negative,5.6,1,161.0,13.0,0
7008,28592,2023-03-31,6.5,PR3,199.0,259.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.70,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


In [162]:
rtx_yes['RKD.ID'].nunique()

115

In [163]:
rtx_no=data.loc[(data['RTX_Status_induction']==0)]
rtx_no

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6962,28502,2018-10-19,11.4,MPO,105.0,378.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,128.0,NaN,0
6963,28502,2019-11-01,23.8,MPO,378.0,581.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,122.0,NaN,0
6964,28502,2021-06-04,42.9,MPO,581.0,504.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,10.0,1,117.0,2.6,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [164]:
rtx_no['RKD.ID'].nunique()

288

## Rituximab dose category

In [165]:
data['RTX_dose_category'].value_counts(dropna=False)

RTX_dose_category
Not received      1936
Rituximab high     246
Dose Unknown       222
Rituximab low      119
Name: count, dtype: int64

In [166]:
rtx_no=data.loc[(data['RTX_dose_category']=='Not received')]
rtx_no

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6962,28502,2018-10-19,11.4,MPO,105.0,378.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,128.0,NaN,0
6963,28502,2019-11-01,23.8,MPO,378.0,581.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,122.0,NaN,0
6964,28502,2021-06-04,42.9,MPO,581.0,504.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,10.0,1,117.0,2.6,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [167]:
rtx_no['RKD.ID'].nunique()

288

In [168]:
rtx_high=data.loc[(data['RTX_dose_category']=='Rituximab high')]
rtx_high

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
602,33,2016-02-02,15.4,MPO,361.0,412.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,62.0,60.0,0
603,33,2017-03-20,28.9,MPO,412.0,73.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,73.0,1.7,0
604,33,2017-06-01,31.3,MPO,73.0,852.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
605,33,2019-10-01,59.3,MPO,852.0,227.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,10.0,NaN,64.0,3.0,0
606,33,2020-05-15,66.8,MPO,227.0,47.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,160.56,Negative,NaN,Negative,61.0,91.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6662,28271,2021-08-03,10.3,PR3,90.0,189.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,Negative,NaN,1,217.0,NaN,0
6663,28271,2022-02-08,16.5,PR3,189.0,147.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,18.0,NaN,189.0,5.5,0
6664,28271,2022-07-05,21.3,PR3,147.0,NaN,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,216.0,2.6,0
6856,28389,2022-08-05,9.5,MPO,168.0,227.0,1,Rituximab high,0,0,...,NaN,No,Off Dialysis,NaN,1,70.0,Negative,112.0,7.6,0


In [169]:
rtx_high['RKD.ID'].nunique()

42

In [170]:
rtx_low=data.loc[(data['RTX_dose_category']=='Rituximab low')]
rtx_low

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
2099,182,2019-01-18,16.1,MPO,357.0,462.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,1,NaN,Negative,NaN,NaN,0
2100,182,2020-04-24,31.3,MPO,462.0,406.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,387.0,NaN,315.0,NaN,0
2101,182,2021-06-04,44.6,MPO,406.0,518.0,1,Rituximab low,0,0,...,NaN,No,On Dialysis,272.1,>=+3,NaN,Negative,999.0,NaN,0
2102,182,2022-11-04,61.7,MPO,518.0,588.0,1,Rituximab low,0,0,...,NaN,No,On Dialysis,108.6,>=+3,91.0,1,999.0,3.0,0
2103,182,2024-06-14,81.0,MPO,588.0,NaN,1,Rituximab low,0,0,...,NaN,No,Transplant,NaN,Negative,2.9,2,134.0,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6947,28481,2023-10-27,10.9,PR3,147.0,181.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,23.6,Negative,1.4,Negative,81.0,1.4,0
6948,28481,2024-04-25,16.9,PR3,181.0,15.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
6949,28481,2024-05-10,17.4,PR3,15.0,NaN,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,32.4,Negative,0.6,Negative,82.0,6.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


In [171]:
rtx_low['RKD.ID'].nunique()

30

In [172]:
rtx_unknown=data.loc[(data['RTX_dose_category']=='Dose Unknown')]
rtx_unknown

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
1409,117,2008-01-07,6.1,MPO,180.0,326.0,1,Dose Unknown,1,0,...,no,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
1410,117,2008-11-28,16.8,MPO,326.0,2464.0,1,Dose Unknown,1,0,...,no,No,Off Dialysis,NaN,1,NaN,Negative,NaN,NaN,0
1411,117,2015-08-28,97.7,MPO,2464.0,175.0,1,Dose Unknown,1,0,...,no,No,Off Dialysis,81.5,1,22.0,2,109.0,2.0,0
1412,117,2016-02-19,103.5,MPO,175.0,161.0,1,Dose Unknown,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,98.0,19.0,0
1413,117,2016-07-29,108.8,MPO,161.0,210.0,1,Dose Unknown,1,0,...,no,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,28546,2023-06-02,9.3,MPO,310.0,278.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,31.3,1,17.0,2,177.0,32.0,0
6993,28546,2024-03-06,18.5,MPO,278.0,100.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,4.0,NaN,147.0,NaN,0
6994,28546,2024-06-14,21.7,MPO,100.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,43.7,Negative,5.6,1,161.0,13.0,0
7008,28592,2023-03-31,6.5,PR3,199.0,259.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


In [173]:
rtx_unknown['RKD.ID'].nunique()

43

## IV Cyclophosphamide : Patients who received IV CYC during induction phase

In [174]:
data['Cyc_Status_induction'].value_counts(dropna=False)

Cyc_Status_induction
0    1467
1    1056
Name: count, dtype: int64

In [175]:
cyc_yes=data.loc[data['Cyc_Status_induction']==1]
cyc_yes

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6962,28502,2018-10-19,11.4,MPO,105.0,378.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,128.0,NaN,0
6963,28502,2019-11-01,23.8,MPO,378.0,581.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,122.0,NaN,0
6964,28502,2021-06-04,42.9,MPO,581.0,504.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,10.0,1,117.0,2.6,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [176]:
cyc_yes['RKD.ID'].nunique()

129

In [177]:
cyc_no=data.loc[data['Cyc_Status_induction']==0]
cyc_no

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
124,3,2012-10-19,6.6,MPO,28.0,28.0,0,Not received,0,1,...,no,No,Off Dialysis,NaN,NaN,23.0,NaN,106.0,5.0,0
125,3,2012-11-16,7.5,MPO,28.0,63.0,0,Not received,0,1,...,no,No,Off Dialysis,NaN,NaN,15.0,NaN,111.0,1.0,0
126,3,2013-01-18,9.6,MPO,63.0,154.0,0,Not received,0,1,...,no,No,Functioning native kidneys,NaN,NaN,11.0,NaN,115.0,1.0,0
127,3,2013-06-21,14.7,MPO,154.0,147.0,0,Not received,0,1,...,no,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,1.0,0
128,3,2013-11-15,19.5,MPO,147.0,238.0,0,Not received,0,1,...,no,No,Off Dialysis,NaN,NaN,23.0,NaN,115.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,28546,2024-06-14,21.7,MPO,100.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,43.7,Negative,5.6,1,161.0,13.0,0
7008,28592,2023-03-31,6.5,PR3,199.0,259.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.7,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


## Oral Cyclophosphamide : Patients who received oral Cyc during induction phase

In [178]:
data['OralCyc_Status_induction'].value_counts(dropna=False)

OralCyc_Status_induction
0    1717
1     806
Name: count, dtype: int64

In [179]:
oral_cyc_yes=data.loc[data['OralCyc_Status_induction']==1]
oral_cyc_yes

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
61,2,2012-10-30,7.0,MPO,39.0,24.0,0,Not received,1,1,...,no,No,Functioning native kidneys,NaN,NaN,NaN,NaN,342.0,1.0,0
62,2,2012-11-23,7.8,MPO,24.0,28.0,0,Not received,1,1,...,no,No,Off Dialysis,NaN,NaN,100.0,NaN,321.0,6.0,0
63,2,2012-12-21,8.7,MPO,28.0,49.0,0,Not received,1,1,...,no,No,Functioning native kidneys,NaN,NaN,36.0,NaN,393.0,1.0,0
64,2,2013-02-08,10.3,MPO,49.0,133.0,0,Not received,1,1,...,no,No,Functioning native kidneys,NaN,NaN,32.0,NaN,346.0,NaN,0
65,2,2013-06-21,14.7,MPO,133.0,98.0,0,Not received,1,1,...,no,No,Off Dialysis,NaN,NaN,32.0,NaN,330.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6761,28325,2020-06-29,260.9,PR3,231.0,168.0,0,Not received,0,1,...,no,Yes,Off Dialysis,NaN,Negative,NaN,Negative,69.0,1.6,1
6775,28330,2021-08-27,6.8,PR3,77.0,189.0,1,Dose Unknown,0,1,...,NaN,No,Off Dialysis,255.4,2,166.0,2,144.0,2.0,0
6776,28330,2022-03-04,13.0,PR3,189.0,392.0,1,Dose Unknown,0,1,...,NaN,No,Off Dialysis,NaN,>=+3,141.0,1,125.0,3.0,0
6777,28330,2023-03-31,25.9,PR3,392.0,448.0,1,Dose Unknown,0,1,...,NaN,No,Off Dialysis,224.0,2,53.0,Negative,140.0,4.0,0


In [180]:
oral_cyc_yes['RKD.ID'].nunique()

116

In [181]:
oral_cyc_no=data.loc[data['OralCyc_Status_induction']==0]
oral_cyc_no

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.7,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7012,28594,2024-01-05,10.0,PR3,18.0,NaN,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,Negative,1.2,Negative,75.0,4.0,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [182]:
oral_cyc_no['RKD.ID'].nunique()

287

## Last 6 months medication details

In [183]:
data['Rituximab_6months'].value_counts(dropna=False)

Rituximab_6months
0    2241
1     282
Name: count, dtype: int64

In [184]:
data['Rituximab_6months'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\2701590674.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Rituximab_6months'].fillna(0,inplace=True)


In [185]:
rtx=data.loc[data['Rituximab_6months']==0]
rtx

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
11,1,2013-03-01,6.0,MPO,35.0,35.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,88.0,NaN,152.0,4.6,0
12,1,2013-04-05,7.1,MPO,35.0,7.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,116.0,NaN,116.0,6.5,0
13,1,2013-04-12,7.4,MPO,7.0,14.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,116.0,NaN,139.0,16.0,0
14,1,2013-04-26,7.8,MPO,14.0,53.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,144.0,NaN,113.0,3.6,0
15,1,2013-06-18,9.6,MPO,53.0,24.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,65.0,NaN,127.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6962,28502,2018-10-19,11.4,MPO,105.0,378.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,128.0,NaN,0
6963,28502,2019-11-01,23.8,MPO,378.0,581.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,NaN,1,122.0,NaN,0
6964,28502,2021-06-04,42.9,MPO,581.0,504.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,10.0,1,117.0,2.6,0
7016,28615,2015-05-19,49.8,MPO,1516.0,647.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,22.0,Negative,157.0,2.3,0


In [186]:
rtx['RKD.ID'].nunique()

384

In [187]:
rtx1=data.loc[data['Rituximab_6months']==1]
rtx1

,RKD.ID,Date_Of_Visit,Interval.from.diagnosis..months._x,At.any.point.ANCA.specificity,Delta_t_previous,Delta_t_post,RTX_Status_induction,RTX_dose_category,Cyc_Status_induction,OralCyc_Status_induction,...,LTROT,Treatment_stopped,Dialysis,Urine.sCD163..ng.mmmol...Euroimmun,Urinalysis.Protein,Urine.PCR.ACR..mg.mmol.,Urinalysis.Blood,Creatinine,CRP,Relapse_next_12_months
224,7,2012-07-06,9.2,PR3,275.0,133.0,0,Not received,1,0,...,no,No,Functioning native kidneys,NaN,NaN,137.0,NaN,171.0,1.8,0
225,7,2012-11-16,13.5,PR3,133.0,216.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,NaN,35.0,NaN,156.0,8.0,0
226,7,2013-06-20,20.6,PR3,216.0,127.0,0,Not received,1,0,...,no,No,Off Dialysis,NaN,Negative,11.0,Negative,152.0,7.0,1
347,17,2013-03-22,19.7,PR3,504.0,1.0,0,Not received,0,1,...,NaN,No,Functioning native kidneys,NaN,NaN,NaN,NaN,NaN,NaN,0
348,17,2013-03-23,19.7,PR3,1.0,174.0,0,Not received,0,1,...,NaN,No,Off Dialysis,NaN,NaN,72.0,NaN,129.0,33.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6994,28546,2024-06-14,21.7,MPO,100.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,43.7,Negative,5.6,1,161.0,13.0,0
7008,28592,2023-03-31,6.5,PR3,199.0,259.0,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0
7009,28592,2023-12-15,15.0,PR3,259.0,NaN,1,Dose Unknown,0,0,...,NaN,No,Off Dialysis,9.7,Negative,3.0,Negative,86.0,16.0,0
7011,28594,2023-12-18,9.4,PR3,286.0,18.0,1,Rituximab low,0,0,...,NaN,No,Off Dialysis,NaN,NaN,NaN,NaN,NaN,NaN,0


## MMF_12months: Patient received MMF last 6 months or not

In [188]:
data['MMF_6months'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3104027457.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['MMF_6months'].fillna(0,inplace=True)


In [189]:
data['MMF_6months'].value_counts(dropna=False)

MMF_6months
0    2263
1     260
Name: count, dtype: int64

## Methotrexate_6months: Patient received Methotrexate last 6 months or not

In [190]:
data['Methotrexate_6months'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3628220244.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Methotrexate_6months'].fillna(0,inplace=True)


In [191]:
data['Methotrexate_6months'].value_counts(dropna=False)

Methotrexate_6months
0    2383
1     140
Name: count, dtype: int64

## AZA_6months

In [192]:
data['AZA_6months'].value_counts(dropna=False)

AZA_6months
0    1500
1    1023
Name: count, dtype: int64

## ANCA Kinetics

In [193]:
 data['anca_kinetics_longterm'].value_counts(dropna=False)

anca_kinetics_longterm
Unknown ANCA level     2096
persistent positive     211
persistent negative     101
Variable ANCA level      92
NaN                      23
Name: count, dtype: int64

In [194]:
data['anca_kinetics_longterm'].fillna('Unknown ANCA level',inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3154568179.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['anca_kinetics_longterm'].fillna('Unknown ANCA level',inplace=True)


## Current medication details

## Rituximab: Number of patients currently on RTX

In [195]:
data['Rituximab'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\3657345097.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Rituximab'].fillna(0,inplace=True)


In [196]:
data['Rituximab'].value_counts(dropna=False)

Rituximab
0    2242
1     281
Name: count, dtype: int64

## MMF: Currently on MMF

In [197]:
data['MMF'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\161205319.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['MMF'].fillna(0,inplace=True)


In [198]:
data['MMF'].value_counts(dropna=False)

MMF
0    2279
1     244
Name: count, dtype: int64

## Methotrexate: Currently on Methotrexate

In [199]:
data['Methotrexate'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\47302126.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Methotrexate'].fillna(0,inplace=True)


In [200]:
data['Methotrexate'].value_counts(dropna=False)

Methotrexate
0    2394
1     129
Name: count, dtype: int64

## AZA

In [201]:
data['AZA'].fillna(0,inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\2430821554.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['AZA'].fillna(0,inplace=True)


In [202]:
data['AZA'].value_counts(dropna=False)

AZA
0    1572
1     951
Name: count, dtype: int64

## ANCA_switch: Current ANCA Switch Status

In [203]:
data['ANCA_switch'].value_counts(dropna=False)

ANCA_switch
Switch Status Unknown    1825
Pos-Pos                   391
Neg-Neg                   224
Neg-Pos                    47
Pos-Neg                    36
Name: count, dtype: int64

## LTROT: Current LTROT status

In [204]:
data['LTROT'].fillna('NA',inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\287952890.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['LTROT'].fillna('NA',inplace=True)


In [205]:
data['LTROT'].value_counts(dropna=False)

LTROT
no     1163
NA     1006
yes     354
Name: count, dtype: int64

## CPD_on_off: Current treatment status

In [206]:
data['CPD_on_off'].value_counts(dropna=False)

CPD_on_off
On Treatment                1590
Off Treatment                714
Manual review                120
Prednisolone<=10              68
Treatment Status Unknown      31
Name: count, dtype: int64

In [207]:
data['CPD_on_off'] = data['CPD_on_off'].replace(
    ['Manual review'], 
    'Treatment Status Unknown'
)

In [208]:
data['CPD_on_off'].value_counts(dropna=False)

CPD_on_off
On Treatment                1590
Off Treatment                714
Treatment Status Unknown     151
Prednisolone<=10              68
Name: count, dtype: int64

## Dialysis

In [209]:
data['Dialysis'].value_counts(dropna=False)

Dialysis
Off Dialysis                  2115
Functioning native kidneys     169
On Dialysis                    163
Transplant                      76
Name: count, dtype: int64

## Urinalysis.Protein

In [210]:
data['Urinalysis.Protein'].value_counts(dropna=False)

Urinalysis.Protein
NaN         1144
Negative     776
1            273
2            220
>=+3         110
Name: count, dtype: int64

In [211]:
## Missing for 45% opf encounters

In [212]:
data['Urinalysis.Protein'].fillna('Unknown',inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\2796926122.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Urinalysis.Protein'].fillna('Unknown',inplace=True)


## Urine.PCR.ACR..mg.mmol.

In [213]:
data['Urine.PCR.ACR..mg.mmol.'].isnull().sum()

1788

## Urinalysis.Blood

In [214]:
data['Urinalysis.Blood'].value_counts(dropna=False)

Urinalysis.Blood
NaN         1146
Negative     988
1            193
2            142
>=+3          54
Name: count, dtype: int64

In [215]:
data['Urinalysis.Blood'].fillna('Unknown',inplace=True)

C:\Users\georgea1\AppData\Local\Temp\ipykernel_25372\356372179.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Urinalysis.Blood'].fillna('Unknown',inplace=True)


In [219]:
data['Creatinine'].isnull().sum()

646

In [220]:
data['CRP'].isnull().sum()

1129

In [216]:
data['Urine.sCD163..ng.mmmol...Euroimmun'].isnull().sum()

2225

In [107]:
data.shape

(2523, 31)

In [221]:
data.columns

Index(['RKD.ID', 'Date_Of_Visit', 'Interval.from.diagnosis..months._x',
       'At.any.point.ANCA.specificity', 'Delta_t_previous', 'Delta_t_post',
       'RTX_Status_induction', 'RTX_dose_category', 'Cyc_Status_induction',
       'OralCyc_Status_induction', 'Rituximab_6months', 'MMF_6months',
       'Methotrexate_6months', 'AZA_6months', 'Rituximab', 'MMF',
       'Methotrexate', 'AZA', 'ANCA_switch', 'anca_kinetics_longterm',
       'CPD_on_off', 'CPD_relapse', 'CPD ISMed', 'LTROT', 'Treatment_stopped',
       'Dialysis', 'Urine.sCD163..ng.mmmol...Euroimmun', 'Urinalysis.Protein',
       'Urine.PCR.ACR..mg.mmol.', 'Urinalysis.Blood', 'Creatinine', 'CRP',
       'Relapse_next_12_months'],
      dtype='object')

In [222]:
data.to_csv('C:/Users/georgea1/OneDrive - Trinity College Dublin/Desktop/First_relapse_12_months_LSTM_final_data_22_08_2024.csv',index=True)
